<a href="https://colab.research.google.com/github/cchio74/forgemaster/blob/master/Lab14_Deep%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data=np.array([0,0,0,1,1,0,1,1],np.float32).reshape(4,2)
x_data

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]], dtype=float32)

In [3]:
y_data=np.array([0,1,1,0],np.float32).reshape(4,1)
y_data

array([[0.],
       [1.],
       [1.],
       [0.]], dtype=float32)

In [4]:
x_data.shape

(4, 2)

In [5]:
x_data.shape[1]

2

In [6]:
y_data.shape[1]

1

In [7]:
input_dim=x_data.shape[1]
units=y_data.shape[1]

input(nxi), output(nxj) i는 인풋 갯수 j는 아웃풋 갯수, n데이터 갯수
W와 b의 갯수는? W는 ixj개, [i,j] b는 j개 [1,j]

# Logistic regression

In [8]:
learning_rate=1e-2

그렇다면 w와 b의 사이즈를 어떻게 정하는가? 일일이 i와 j의 크기를 넣어줘야 하나? 아니면 자동으로 정할 수 있는가?
x_data와 y_data의 dimension을 뽑을수 있으면 가능함.
- x_data.shape[숫자] : 해당숫자-1에 해당하는 dimension을 뽑음.

In [9]:
w_init=tf.random_normal_initializer()
b_init=tf.zeros_initializer()

In [10]:
w1=tf.Variable(initial_value=[[5,-7],[5,-7]],dtype='float32')
b1=tf.Variable(initial_value=[[-8,3]],dtype='float32')
w2=tf.Variable(initial_value=[[-11],[-11]],dtype='float32')
b2=tf.Variable(initial_value=[[6]],dtype='float32')

In [11]:
for i in range(10000):
  # loss
  with tf.GradientTape() as tape:
    z1=tf.matmul(x_data,w1)+b1   #매트릭스 곱하기 : np.matmul()
    k1=tf.divide(1,1+tf.math.exp(-z1)) #sigmoid
    z2=tf.matmul(k1,w2)+b2
    hypothesis=tf.divide(1,1+tf.math.exp(-z2))
    
    loss=-tf.reduce_mean(y_data*tf.math.log(hypothesis)+(1-y_data)*tf.math.log(1-hypothesis)) #cross entropy

  # gradient
  w1_grad,b1_grad,w2_grad,b2_grad=tape.gradient(loss,[w1,b1,w2,b2])
  

  # updata
  w1.assign_sub(learning_rate*w1_grad)
  b1.assign_sub(learning_rate*b1_grad)
  w2.assign_sub(learning_rate*w2_grad)
  b2.assign_sub(learning_rate*b2_grad)
  

  # display
  if i % 1000==0:
    print('iteration {} loss {}'.format(i,loss))

iteration 0 loss 0.011523084715008736
iteration 1000 loss 0.010212266817688942
iteration 2000 loss 0.009532989002764225
iteration 3000 loss 0.009119556285440922
iteration 4000 loss 0.008840661495923996
iteration 5000 loss 0.008637221530079842
iteration 6000 loss 0.008479657582938671
iteration 7000 loss 0.008350270800292492
iteration 8000 loss 0.008239127695560455
iteration 9000 loss 0.008141405880451202


In [12]:
    z1=tf.matmul(x_data,w1)+b1   #매트릭스 곱하기 : np.matmul()
    k1=tf.divide(1,1+tf.math.exp(-z1)) #sigmoid
    z2=tf.matmul(k1,w2)+b2
    hypothesis=tf.divide(1,1+tf.math.exp(-z2))

    hypothesis.numpy()

array([[0.00734875],
       [0.99245924],
       [0.99245924],
       [0.00964699]], dtype=float32)

In [13]:
w.numpy()

NameError: ignored

In [ ]:
b.numpy()

In [ ]:
w

In [ ]:
tf.divide(1,1+tf.math.exp(-z))

In [ ]:
y_data

# subclassing

In [31]:
# model
class Model(tf.keras.layers.Layer):
  def __init__(self,input_dim,units):
    super(Model,self).__init__()

    self.w1=self.add_weight(shape=(input_dim,2),initializer='random_normal')
    self.b1=self.add_weight(shape=(2,),initializer='zeros')

    self.w2=self.add_weight(shape=(2,units),initializer='random_normal')
    self.b2=self.add_weight(shape=(units,),initializer='zeros')

    self.w1=tf.Variable(initial_value=[[5,-7],[5,-7]],dtype='float32')
    self.b1=tf.Variable(initial_value=[[-8,3]],dtype='float32')
    self.w2=tf.Variable(initial_value=[[-11],[-11]],dtype='float32')
    self.b2=tf.Variable(initial_value=[[6]],dtype='float32')

  def call(self,inputs):
    z1=tf.matmul(inputs,self.w1)+self.b1
    k1=tf.math.sigmoid(z1)
    z2=tf.matmul(k1,self.w2)+self.b2

    return tf.math.sigmoid(z2)

In [32]:
# loss
loss=tf.keras.losses.BinaryCrossentropy()   #yes no만 있는 crossentropy

In [33]:
# gradient
def grad(model,inputs,targets):
  with tf.GradientTape() as tape:
    loss_value=loss(targets,model(inputs))
  return tape.gradient(loss_value,[model.w1, model.b1,model.w2, model.b2])

In [34]:
# define
model=Model(input_dim,units)
optimizer=tf.keras.optimizers.SGD(learning_rate)

In [35]:
# learning
for i in range(10000):
  # gradient
  grads=grad(model,x_data,y_data)

  # updata
  optimizer.apply_gradients(zip(grads,[model.w1,model.b1,model.w2, model.b2]))

  # display

  if i % 2000==0:
    print('{} {}'.format(i,loss(y_data,model(x_data))))


0 0.011521052569150925
2000 0.0095323221758008
4000 0.008840239606797695
6000 0.008479312062263489
8000 0.00823887251317501


In [37]:
print(model.w1.numpy(),model.b1.numpy())

[[ 5.1980047 -7.0336523]
 [ 5.1980047 -7.0336523]] [[-7.90591    3.0396836]]


In [38]:
print(model.w2.numpy(),model.b2.numpy())

[[-11.282783]
 [-11.200979]] [[5.787794]]


In [ ]:
model.w.numpy()

In [ ]:
model.b.numpy()

In [ ]:
model(x_data)

In [ ]:
y_data

# Sequential model

In [ ]:
# model
model=tf.keras.Sequential()

model.add(tf.keras.Input(shape=(input_dim,)))

model.add(tf.keras.layers.Dense(2,activation='sigmoid',kernel_initializer='random_normal'))
model.add(tf.keras.layers.Dense(units,activation='sigmoid',kernel_initializer='random_normal'))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate),loss=tf.keras.losses.BinaryCrossentropy())

In [ ]:
model.set_weights([np.array([[5,-7],[5,-7]],np.float32),
                   np.array([-8,3],np.float32),
                   np.array([[-11],[-11]],np.float32),
                   np.array([6],np.float32)])

In [ ]:
# learning
model.fit(x_data,y_data,epochs=10000,verbose=0)

In [ ]:
model.evaluate(x_data,y_data,verbose=1)

In [ ]:
model.get_weights()

In [ ]:
y_data

In [ ]:
model.predict(x_data)